In [ ]:
import tensorflow as tf

tf.config.list_physical_devices('GPU')

In [ ]:
seed = 1
batch_size = 32
img_size = 224

data_dir = "train_images"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.1,
    subset="training",
    seed=seed,
    image_size=(img_size, img_size),
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.1,
    subset="validation",
    seed=seed,
    image_size=(img_size, img_size),
    batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.show()

In [ ]:
from tensorflow.data import experimental

AUTOTUNE = experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from tensorflow.keras import applications, layers
from tensorflow.keras.layers.experimental import preprocessing

IMG_SHAPE = (img_size, img_size, 3)
num_classes = len(class_names)

base_model = applications.ResNet152V2(include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    preprocessing.Rescaling(1./255, input_shape=IMG_SHAPE),
    preprocessing.RandomFlip("horizontal_and_vertical"),
    preprocessing.RandomRotation(0.2),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
baseline_history = model.fit(
    train_ds,
    batch_size=batch_size,
    validation_data=val_ds,
    epochs=30
)

In [ ]:
def plot_history(histories):
    plt.figure(figsize=(10, 10))
    
    plt.subplot(2, 1, 1)
    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_loss'], '--', label=name.title() + ' Val')
        plt.plot(history.epoch, history.history['loss'], color=val[0].get_color(), label=name.title() + ' Train')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(2, 1, 2)
    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_accuracy'], '--', label=name.title() + ' Val')
        plt.plot(history.epoch, history.history['accuracy'], color=val[0].get_color(), label=name.title() + ' Train')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.xlim([0, max(history.epoch)])

In [ ]:
plot_history([('baseline', baseline_history)])

In [ ]:
import numpy as np

for images, labels in val_ds.take(1):
    predictions = model.predict(images)
    for i in range(len(predictions)):
        prediction = np.argmax(predictions[i])
        plt.title(f"{class_names[prediction]} {np.max(predictions[i]) * 100:.2f}% ({class_names[labels[i]]})")
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.show()